In [169]:
import json
from pathlib import Path

import astropy.uncertainty as unc
import astropy.units as u
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
from astropy.modeling.models import Linear1D, Gaussian1D, Const1D
from astropy.modeling.fitting import LMLSQFitter, TRFLSQFitter
from astropy.time import Time

from feadme.parser import Template, Parameter, Line, Shape, Disk, Mask

finfo = np.finfo(float)
ERR = 1e-10

In [ ]:
disk_profile = Disk(
    name="halpha_disk",
    center=Parameter(name="center", distribution="normal", low=6562.819 - 5, high=6562.819 + 5, loc=6562.819, scale=2),
    inner_radius=Parameter(name="inner_radius", distribution="log_uniform", low=1e2, high=5e3, loc=1e3, scale=1e1),
    delta_radius=Parameter(name="delta_radius", distribution="log_uniform", low=1e2, high=1e4, loc=5e3, scale=1e3),
    inclination=Parameter(name="inclination", distribution="normal", low=0, high=np.pi / 2, loc=np.pi / 4, scale=np.pi / 8),
    sigma=Parameter(name="sigma", distribution="log_uniform", low=2e2, high=1e4, loc=1e3, scale=1e1),
    q=Parameter(name="q", distribution="uniform", low=0.5, high=4, loc=2, scale=1),
    eccentricity=Parameter(name="eccentricity", distribution="uniform", low=0, high=1, loc=0.5, scale=0.25),
    apocenter=Parameter(name="apocenter", distribution="circular", low=0, high=2 * np.pi, loc=np.pi, scale=np.pi / 2),
    scale=Parameter(name="scale", distribution="uniform", low=0, high=1, loc=0.5, scale=0.1),
    offset=Parameter(name="offset", distribution="uniform", low=0, high=0.05, fixed=True, value=0)
)

halpha_narrow_line = Line(
    name="halpha_narrow",
    shape=Shape.gaussian,
    center=Parameter(name="center", distribution="normal", low=6548, high=6568, loc=6562.819, scale=2, shared="halpha_disk"),
    amplitude=Parameter(name="amplitude", distribution="uniform", low=0, high=1, loc=0.5, scale=1),
    vel_width=Parameter(name="vel_width", distribution="log_uniform", low=1e1, high=1e3, loc=1e2, scale=1e2),
)

halpha_broad_line = Line(
    name="halpha_broad",
    shape=Shape.gaussian,
    center=Parameter(name="center", distribution="normal", low=6548, high=6568, loc=6562.819, scale=2, shared="halpha_disk"),
    amplitude=Parameter(name="amplitude", distribution="uniform", low=0, high=1, loc=0.5, scale=1),
    vel_width=Parameter(name="vel_width", distribution="log_uniform", low=1e3, high=5e4, loc=5e3, scale=1e2),
)

hei_narrow_line = Line(
    name="hei_narrow",
    shape=Shape.gaussian,
    center=Parameter(name="center", distribution="normal", low=6673, high=6683, loc=6678, scale=2),
    amplitude=Parameter(name="amplitude", distribution="uniform", low=0, high=1, loc=0.5, scale=1),
    vel_width=Parameter(name="vel_width", distribution="log_uniform", low=1e1, high=5e3, loc=1e2, scale=1e1),
)

un_narrow_line = Line(
    name="unknown_narrow",
    shape=Shape.gaussian,
    center=Parameter(name="center", distribution="uniform", low=6225, high=6330, loc=6230, scale=2),
    amplitude=Parameter(name="amplitude", distribution="uniform", low=0, high=1, loc=0.5, scale=1),
    vel_width=Parameter(name="vel_width", distribution="log_uniform", low=1e1, high=1e4, loc=5e2, scale=1e1),
)

abs_narrow_line = Line(
    name="absorption_narrow",
    shape=Shape.gaussian,
    center=Parameter(name="center", distribution="uniform", low=6650, high=6725, loc=6680, scale=2),
    amplitude=Parameter(name="amplitude", distribution="uniform", low=-1, high=0, loc=-0.5, scale=1),
    vel_width=Parameter(name="vel_width", distribution="log_uniform", low=1e1, high=5e3, loc=5e2, scale=1e1),
)

siir_narrow_line = Line(
    name="siir_narrow",
    shape=Shape.gaussian,
    center=Parameter(name="center", distribution="normal", low=6730.81 - 5, high=6730.81 + 5, loc=6730.81, scale=2),
    amplitude=Parameter(name="amplitude", distribution="uniform", low=0, high=1, loc=0.5, scale=1),
    vel_width=Parameter(name="vel_width", distribution="log_uniform", low=1e1, high=1e3, loc=1e2, scale=1e1, shared="halpha_narrow"),
)

siil_narrow_line = Line(
    name="siil_narrow",
    shape=Shape.gaussian,
    center=Parameter(name="center", distribution="normal", low=6716.44 - 5, high=6716.44 + 5, loc=6716.44, scale=2),
    amplitude=Parameter(name="amplitude", distribution="uniform", low=0, high=1, loc=0.5, scale=1),
    vel_width=Parameter(name="vel_width", distribution="log_uniform", low=1e1, high=1e3, loc=1e2, scale=1e1, shared="siir_narrow"),
)

white_noise=Parameter(name="white_noise", distribution="uniform", low=-10, high=1)

In [171]:
Path.home() / Path("research/tde_agn_comparison/parsed")

PosixPath('/Users/nmearl/research/tde_agn_comparison/parsed')

In [ ]:
ingest_dir = Path.home() / Path("research/tde_agn_comparison/parsed")
output_dir = Path.home() / Path("research/tde_agn_comparison")

for file_path in sorted(ingest_dir.glob("*.ecsv")):
    for seg in ['2018hyz', '2018zr', '09djl', '2020zso']:#'14li', '2018hyz', '2018zr',]:#, '2020zso', '09djl']:
        if seg not in file_path.stem:
            print(f"Could not find {seg}")
            continue
    
        tab = Table.read(file_path, format='ascii.ecsv')
        label = tab.meta['name'].replace(',', '').replace(' ', '')
        redshift = tab.meta['redshift']
        mjd = tab.meta['obs_date']
        
        wave = tab['wave'] * u.AA
        flux = tab['flux'] * u.erg / u.s / u.cm ** 2 / u.AA
        flux_err = tab['uncertainty'] * u.erg / u.s / u.cm ** 2 / u.AA
        rest_wave = wave / (1 + redshift)
        rest_vel = rest_wave.to(u.km / u.s, u.doppler_optical(6563 * u.AA))
        vel_mask = (rest_vel.value >= -15e3) & (rest_vel.value <= +15e3)
        print(rest_vel)
        minw, maxw = np.min(rest_wave.value[vel_mask]), np.max(rest_wave.value[vel_mask])

        flux = flux.to(u.mJy, u.spectral_density(wave))
        flux_err = flux_err.to(u.mJy, u.spectral_density(wave))

        ha_mask = (rest_wave.value > 6563 - 350) & (rest_wave.value < 6563 + 350)
        full_mask = (rest_wave.value > 6563 - 650 * 2) & (rest_wave.value < 6563 + 650 * 2)
        part_mask = (rest_wave.value > 6563 - 300) & (rest_wave.value < 6563 + 300)
        cont_mask = full_mask & ~part_mask
        edge_mask = ((rest_wave.value > 6010) & (rest_wave.value < 6400))

        g = Gaussian1D(mean=6270, amplitude=0.5, stddev=15, bounds={"mean": (6270-25, 6270+50),
                                                                    "amplitude": (0, None),
                                                                    "stddev": (10, None)
                                                                     })
        fit_mod = TRFLSQFitter()(Const1D() + g, rest_wave[edge_mask], flux[edge_mask])

        cont_flux = fit_mod(rest_wave)
        # 
        # fig, ax = plt.subplots(figsize=(12, 4))
        #
        # ax.axvline(fit_mod[1].mean.value, label=f"{fit_mod[1].mean.value:.3f}, {fit_mod[1].stddev.value:.3f}")
        # ax.scatter(rest_wave[edge_mask], flux[edge_mask], label='Data', color='C2', zorder=10)
        # ax.plot(rest_wave[full_mask], flux[full_mask], label='Data')
        # ax.scatter(rest_wave[ha_mask], flux[ha_mask], label='Data')
        # ax.axhline(0)
        # ax.axvline(minw)
        # ax.axvline(maxw)
        # ax.set_title(f"{label} {mjd - 58428} {mjd}")
        # ax.plot(rest_wave[full_mask], cont_flux[full_mask], label='Continuum')
        # ax.legend()
        # 
        # flux -= cont_flux

        # ax.plot(rest_wave[full_mask], flux[full_mask], label='Continuum Subtracted')
        # ax.set_title(f"{label} {Time(mjd, format='mjd').iso}")
        
        # flux = flux[ha_mask]
        # flux_err = flux_err[ha_mask]
        # wave = wave[ha_mask]
    
        Table({'wave': wave, 'flux': flux, 'flux_err': flux_err}).write(
            output_dir / "data" / f"{label}_{mjd}.csv", format='ascii.csv', overwrite=True)
        
        template = Template(
            name=label,
            redshift=redshift,
            mjd=mjd,
            data_path=str(output_dir / "data" / f"{label}_{mjd}.csv"),
            # data_path=f"/datasets/transient-data/{label}_{mjd}.csv",
            # data_path=f"/home/kasm-user/research/data/{label}_{mjd}.csv",
            disk_profiles=[
                disk_profile,
            ],
            line_profiles=[],
            white_noise=white_noise,
            mask=[Mask(lower_limit=6563 - 350, upper_limit=6563 + 350)]
        )
        
        if '2018zr' in seg:
            template.redshift = 0.071
            halpha_broad_line_copy = halpha_broad_line.model_copy(deep=True)
            halpha_broad_line_copy.vel_width.low = 1e3
            halpha_broad_line_copy.vel_width.high = 1e4
            template.line_profiles=[
                # halpha_broad_line_copy,
                abs_narrow_line
                # siil_narrow_line,
                # siir_narrow_line
            ]

            if mjd == 58221:
                template.line_profiles.append(un_narrow_line)
                template.mask = [
                    Mask(lower_limit=6100, upper_limit=6200),
                    Mask(lower_limit=6350, upper_limit=6563 + 350)
                ]

        elif '2020zso' in seg:
            halpha_broad_line_copy = halpha_broad_line.model_copy(deep=True)
            halpha_broad_line_copy.vel_width.high = 5e3
            template.line_profiles=[
                # halpha_broad_line_copy,
            ]
            # template.mask = [
            #     Mask(lower_limit=6150, upper_limit=6530),
            #     Mask(lower_limit=6600, upper_limit=7200)
            # ]
            template.mask = [
                Mask(lower_limit=6200, upper_limit=7200)
            ]
        elif '2018hyz' in seg:
            template.line_profiles=[
                # halpha_broad_line,
            ]
            if 58460 < mjd <= 58497:
                template.line_profiles.append(un_narrow_line)

        elif '14li' in seg:
            template.line_profiles=[
                # halpha_narrow_line,
                # halpha_broad_line,
                hei_narrow_line
            ]
        elif '09djl' in seg:
            template.line_profiles=[
                # halpha_broad_line
                ]
    
        for prof in template.disk_profiles:
            prof.scale.high = flux.max().value * 1.1

            for param in prof._independent():
                if "log" in param.distribution:
                    param.scale = 10 ** ((np.log10(param.high) 
                                        - np.log10(param.low)) / 2)
                else:
                    param.scale = ((param.high - param.low) / 2)
    
        for prof in template.line_profiles:
            prof.amplitude.high = flux.max().value * 1.1

            for param in prof._independent():
                if "log" in param.distribution:
                    param.scale = 10 ** ((np.log10(param.high) 
                                        - np.log10(param.low)) / 2)
                else:
                    param.scale = ((param.high - param.low) / 2)
    
        with open(output_dir / "templates" / f"{label}_{mjd}.json", "w") as f:
            json.dump(template.model_dump(), f, indent=4)

Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hyz
Could not find 2018zr
Could not find 09djl
Could not find 2020zso
Could not find 2018hy

In [173]:
np.nan or 10

nan